# HW3

### Problem 1

(Interval Bound Propagation, Programming). In this problem, you will 

implement interval bound propagation (IBP) training for a simple neural network.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64
np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.f1 = nn.Linear(28*28, 50)
        self.f2 = nn.Linear(50, 50)
        self.f3 = nn.Linear(50, 50)
        self.out = nn.Linear(50, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        x = F.relu(self.f3(x))
        x = self.out(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Sequential(
  (0): Normalize()
  (1): Net(
    (f1): Linear(in_features=784, out_features=50, bias=True)
    (f2): Linear(in_features=50, out_features=50, bias=True)
    (f3): Linear(in_features=50, out_features=50, bias=True)
    (out): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [3]:
def interval_analysis_vectorized(model, x, eps):

    model = model.to(device)
    
    f1_weight = model[1].f1.weight
    f1_bias = model[1].f1.bias
    f2_weight = model[1].f2.weight
    f2_bias = model[1].f2.bias
    f3_weight = model[1].f3.weight
    f3_bias = model[1].f3.bias
    out_weight = model[1].out.weight
    out_bias = model[1].out.bias

    x = x.view(x.size(0), -1)  
    x_lower = x - eps
    x_upper = x + eps

    # Layer 1
    W_pos = torch.clamp(f1_weight, min=0)
    W_neg = torch.clamp(f1_weight, max=0)
    f1_lower = torch.matmul(x_lower, W_pos.t()) + torch.matmul(x_upper, W_neg.t()) + f1_bias
    f1_upper = torch.matmul(x_upper, W_pos.t()) + torch.matmul(x_lower, W_neg.t()) + f1_bias
    f1_lower = F.relu(f1_lower)
    f1_upper = F.relu(f1_upper)

    # Layer 2
    W_pos = torch.clamp(f2_weight, min=0)
    W_neg = torch.clamp(f2_weight, max=0)
    f2_lower = torch.matmul(f1_lower, W_pos.t()) + torch.matmul(f1_upper, W_neg.t()) + f2_bias
    f2_upper = torch.matmul(f1_upper, W_pos.t()) + torch.matmul(f1_lower, W_neg.t()) + f2_bias
    f2_lower = F.relu(f2_lower)
    f2_upper = F.relu(f2_upper)

    # Layer 3
    W_pos = torch.clamp(f3_weight, min=0)
    W_neg = torch.clamp(f3_weight, max=0)
    f3_lower = torch.matmul(f2_lower, W_pos.t()) + torch.matmul(f2_upper, W_neg.t()) + f3_bias
    f3_upper = torch.matmul(f2_upper, W_pos.t()) + torch.matmul(f2_lower, W_neg.t()) + f3_bias
    f3_lower = F.relu(f3_lower)
    f3_upper = F.relu(f3_upper)

    # Output Layer
    W_pos = torch.clamp(out_weight, min=0)
    W_neg = torch.clamp(out_weight, max=0)
    out_lower = torch.matmul(f3_lower, W_pos.t()) + torch.matmul(f3_upper, W_neg.t()) + out_bias
    out_upper = torch.matmul(f3_upper, W_pos.t()) + torch.matmul(f3_lower, W_neg.t()) + out_bias

    return out_lower, out_upper

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import time

def train_ibp(model, train_loader,
             num_epochs=20, learning_rate=0.01, momentum=0.9,
             initial_kappa=1.0, final_kappa=0.5,
             initial_epsilon=0.0, final_epsilon=0.1):

    model = model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    kappa = initial_kappa
    decay = (initial_kappa - final_kappa) / num_epochs
    epsilon = initial_epsilon
    epsilon_increment = (final_epsilon - initial_epsilon) / num_epochs

    model.train()
    start_time = time.time()
    
    for epoch in range(1, num_epochs + 1):
        epoch_loss = 0.0
        
        kappa = max(kappa - decay, final_kappa)
        epsilon = min(epsilon + epsilon_increment, final_epsilon)
        
        for data, target in tqdm(train_loader, desc=f'IBP Training Epoch {epoch}/{num_epochs}'):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            outputs = model(data)
            ce_loss = nn.CrossEntropyLoss()(outputs, target)
            out_lower, out_upper = interval_analysis_vectorized(model, data, epsilon)
            z_hat = out_lower
            z_hat[target] = out_upper[target]
            ce_loss_hat = nn.CrossEntropyLoss()(z_hat, target)
            total_loss = kappa * ce_loss + (1 - kappa) * ce_loss_hat
            total_loss.backward()
            optimizer.step()
            epoch_loss += total_loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        print(f'Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}, kappa: {kappa:.4f}, epsilon: {epsilon:.4f}')
    
    training_time = time.time() - start_time
    print(f'IBP Training completed in {training_time:.2f} seconds')
    return training_time

In [5]:
def train_standard(model, train_loader, num_epochs=20, learning_rate=0.01, momentum=0.9):
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    
    model.train()
    start_time = time.time()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for data, target in tqdm(train_loader, desc=f'Standard Training Epoch {epoch+1}/{num_epochs}'):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = nn.CrossEntropyLoss()(outputs, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')
    training_time = time.time() - start_time
    print(f'Standard Training completed in {training_time:.2f} seconds')
    return training_time

In [13]:
def evaluate_standard_accuracy(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc='Evaluating Standard Accuracy'):
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100.0 * correct / total
    print(f'Standard Accuracy: {accuracy:.2f}%')
    return accuracy

In [7]:
def pgd_untargeted(model, x, y, k=10, eps=0.1, eps_step=0.01):

    model.eval()
    x_prime = x.clone().detach()
    x_prime += 2 * eps * (torch.rand_like(x_prime) - 0.5)
    x_prime = torch.clamp(x_prime, torch.clamp(x - eps, 0, 1), torch.clamp(x + eps, 0, 1))

    for _ in range(k):
        x_prime.requires_grad = True
        output = model(x_prime)
        loss = F.cross_entropy(output, y)
        model.zero_grad()
        loss.backward()
        gradient = x_prime.grad.data
        x_prime = x_prime + eps_step * gradient.sign()
        x_prime = torch.max(torch.min(x_prime, x + eps), x - eps)
        x_prime = torch.clamp(x_prime, 0, 1).detach()

    return x_prime


In [18]:
def evaluate_robust_accuracy(model, test_loader, epsilon=0.1, alpha=0.02, iters=40):
    model.eval()
    correct = 0
    total = 0
    for data, target in tqdm(test_loader, desc='Evaluating Robust Accuracy'):
        data, target = data.to(device), target.to(device)
        adv_data = pgd_untargeted(model, data, target, eps=epsilon, k=iters, eps_step=alpha)
        outputs = model(adv_data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
    accuracy = 100.0 * correct / total
    print(f'Robust Accuracy (PGD, epsilon={epsilon}): {accuracy:.2f}%')
    return accuracy

In [21]:
model_ibp = nn.Sequential(Normalize(), Net()).to(device)
model_ibp.train()

ibp_training_time = train_ibp(model_ibp, train_loader, num_epochs=10, learning_rate=0.01, momentum=0.9, 
                              initial_kappa=1.0, final_kappa=0.5, initial_epsilon=0.0, final_epsilon=0.1)

IBP Training Epoch 1/10:   0%|          | 0/938 [00:00<?, ?it/s]


NameError: name 'margin' is not defined

In [10]:
model_standard = nn.Sequential(Normalize(), Net()).to(device)

model_standard.train()

standard_training_time = train_standard(model_standard, train_loader, num_epochs=10, learning_rate=0.01, momentum=0.9)

Standard Training Epoch 1/10: 100%|██████████| 938/938 [00:17<00:00, 54.94it/s]


Epoch 1/10, Loss: 0.4310


Standard Training Epoch 2/10: 100%|██████████| 938/938 [00:11<00:00, 82.77it/s] 


Epoch 2/10, Loss: 0.1469


Standard Training Epoch 3/10: 100%|██████████| 938/938 [00:10<00:00, 85.47it/s] 


...

Epoch 8/10, Loss: 0.0524


Standard Training Epoch 9/10: 100%|██████████| 938/938 [00:24<00:00, 38.32it/s]


Epoch 9/10, Loss: 0.0455


Standard Training Epoch 10/10: 100%|██████████| 938/938 [00:24<00:00, 37.75it/s] 

Epoch 10/10, Loss: 0.0404
Standard Training completed in 159.10 seconds


In [19]:
print("Evaluating IBP Trained Model:")
ibp_standard_acc = evaluate_standard_accuracy(model_ibp, test_loader)
ibp_robust_acc = evaluate_robust_accuracy(model_ibp, test_loader, epsilon=0.1, alpha=0.02, iters=40)

print("Evaluating Standard Trained Model:")
standard_standard_acc = evaluate_standard_accuracy(model_standard, test_loader)
standard_robust_acc = evaluate_robust_accuracy(model_standard, test_loader, epsilon=0.1, alpha=0.02, iters=40)

Evaluating IBP Trained Model:


Evaluating Standard Accuracy:   0%|          | 0/10000 [00:00<?, ?it/s]

Evaluating Standard Accuracy: 100%|██████████| 10000/10000 [00:01<00:00, 6983.72it/s]


Standard Accuracy: 97.38%


Evaluating Robust Accuracy: 100%|██████████| 10000/10000 [01:44<00:00, 95.43it/s]


Robust Accuracy (PGD, epsilon=0.1): 8.22%
Evaluating Standard Trained Model:


Evaluating Standard Accuracy: 100%|██████████| 10000/10000 [00:01<00:00, 8231.65it/s]


Standard Accuracy: 96.69%


Evaluating Robust Accuracy:  35%|███▍      | 3454/10000 [00:36<01:09, 94.14it/s] 


KeyboardInterrupt: 

In [ ]:
print(f"\nTraining Time Comparison:")
print(f"IBP Training Time: {ibp_training_time:.2f} seconds")
print(f"Standard Training Time: {standard_training_time:.2f} seconds")

In [ ]:
epsilon_tests = np.linspace(0.01, 0.1, 10)

model_ibp.eval()
verified_counts = {epsilon: 0 for epsilon in epsilon_tests}
total = 0


for data, target in tqdm(test_loader):
    data, target = data.to(device), target.to(device)
    total += 1
    out_lower, out_upper = interval_analysis_vectorized(model_ibp, data, 0.0) 
    _, predicted = torch.max(out_lower, 1)
    true_class = target.item()

    for epsilon in epsilon_tests:
        out_lower_eps, out_upper_eps = interval_analysis_vectorized(model_ibp, data, epsilon)
        
        true_low = out_lower_eps[true_class]
        other_upper = out_upper_eps.clone()
        other_upper[true_class] = -float('inf')
        
        if true_low > torch.max(other_upper):
            verified_counts[epsilon] += 1

verified_accuracies = {epsilon: 100.0 * count / total for epsilon, count in verified_counts.items()}


# Plot verified accuracy vs epsilon
plt.figure(figsize=(8,6))
plt.plot(epsilon_tests, [verified_accuracies[e] for e in epsilon_tests], marker='o')
plt.title('Verified Accuracy vs Epsilon')
plt.xlabel('Epsilon')
plt.ylabel('Verified Accuracy (%)')
plt.grid(True)
plt.show()

# Print verified accuracies
for epsilon in epsilon_tests:
    print(f"Epsilon: {epsilon:.2f}, Verified Accuracy: {verified_accuracies[epsilon]:.2f}%")

In [ ]:
def find_adversarial_examples(model, test_loader, epsilon_tests):
    model.eval()
    adversarial_examples = {epsilon: [] for epsilon in epsilon_tests}
    for data, target in tqdm(test_loader, desc='Finding Adversarial Examples'):
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        true_class = target.item()
        for epsilon in epsilon_tests:
            out_lower, out_upper = interval_analysis_vectorized(model, data, epsilon)
            true_lower = out_lower[0, true_class]
            other_upper = out_upper.clone()
            other_upper[0, true_class] = -float('inf')
            if not (true_lower > other_upper).all():
                adv_data = pgd_untargeted(model, data, target)
                adversarial_examples[epsilon].append((data.cpu(), adv_data.cpu(), target.cpu()))
                if len(adversarial_examples[epsilon]) >= 5:
                    break
    return adversarial_examples
epsilon_tests = np.linspace(0.01, 0.1, 10)
adversarial_examples = find_adversarial_examples(model_ibp, test_loader, epsilon_tests)


In [ ]:
# Visualize adversarial examples for a specific epsilon
epsilon_tests = np.linspace(0.01, 0.1, 10)

for selected_epsilon in epsilon_tests:
    if adversarial_examples:
        num_examples = min(2, len(adversarial_examples))
        plt.figure(figsize=(15, 10))
        print(f"Adversarial Examples for epsilon={selected_epsilon}")
        print("num_examples", num_examples)
        print(len(adversarial_examples))
        for i in range(num_examples):
            orig, adv, target = adversarial_examples[selected_epsilon][i]
            orig_img = orig.squeeze().numpy()
            adv_img = adv.squeeze().numpy()
            
            plt.subplot(num_examples, 2, 2*i+1)
            plt.imshow(orig_img, cmap='gray')
            plt.title(f'Original: {target.item()}')
            plt.axis('off')

            plt.subplot(num_examples, 2, 2*i+2)
            plt.imshow(adv_img, cmap='gray')
            plt.title('Adversarial')
            plt.axis('off')

        plt.tight_layout()
        plt.show()
    else:
        print(f"No adversarial examples found for epsilon={selected_epsilon}")

## Problem 2

See colab link in the home work file